# Historical Data Collection

In this notebook, I am exploring how I can collect historical price, news and other data for a given stock. I will be looking at Alpha Vantage and the Reddit API to start with. I will also store the data locally in a MySQL database before migrating it to a cloud database on Digital Ocean.

In [ ]:
import pandas as pd
import requests
import json
import collection_functions as c
import mysql.connector

## Alpha Vantage API for Price Data

Alpha Vantage allows 25 API calls per day for free so I will be using this sparingly for now to collect and store as much historical data as possible.

In [ ]:
av_api_key = json.load(open('../../secrets.json'))['alpha_vantage']
db_password = json.load(open('../../secrets.json'))['database_password']

Each request takes in a series of parameters that determine the stock, the type of data, the type of output and the granularity -- among other things. The full API documentation can be found [here](https://www.alphavantage.co/documentation/).

Initially, I look at the TIME_SERIES_INTRADAY endpoint for the S&P 500 at the 1 minute level.

In [ ]:
request_params = {
    'function' : 'TIME_SERIES_INTRADAY',
    'symbol' : 'SPY',
    'interval' : '1min',
    'outputsize' : 'compact',
    'datatype' : 'json',
    'apikey' : av_api_key,
}

url = c.get_av_request_url(request_params) # This is a function from collection_functions.py that builds the URL for the API request from the parameters.
print(url)

We then perform the request which returns a JSON object with some metadata about the request and the time series data itself. The time series data is a dictionary with the time as the key and the OHLCV data as the value.

In [ ]:
r = requests.get(url)
data = r.json()

Next up, we need to transform the data in order to insert it into the database. First, we convert it to a pandas dataframe before converting each column to the appropriate data type.

In [ ]:
df = pd.DataFrame(data['Time Series (1min)']).T # Convert the JSON to pandas, and then transpose it so that the timestamps are the index.

df['interval'] = data['Meta Data']['4. Interval'] # Create a new column for the interval, which is constant for this request.

df.rename(columns={'1. open' : 'open_price', '2. high' : 'high_price',
                   '3. low' : 'low_price', '4. close' : 'close_price',
                   '5. volume' : 'volume'}, inplace=True) # Rename the columns for the database (not strictly necessary, but makes it clearer).

# Convert the numeric columns to float, as they are currently strings.
df[['open_price', 'high_price', 'low_price', 'close_price', 'volume']] = df[['open_price', 'high_price', 'low_price', 'close_price', 'volume']].astype(float)

# set the index as its own column and convert to a timestamp
df['close_timestamp'] = pd.to_datetime(df.index)
df.reset_index(drop=True, inplace=True)

The next stage is to get the stock_id for the S&P 500 from the database so that can add it to our dataframe before inserting it into the database. We do this with a `SELECT FROM WHERE` query to get the stock_id for the 'SPY' ticker symbol. We then add a new column to the dataframe with the stock_id.

In order to recreate this you will need to have a MySQL database set up locally with the schema defined in the `/database/DDL.sql` file. Adjust the database connection parameters as necessary.

In [ ]:
symbol = data['Meta Data']['2. Symbol'] # Get the stock symbol from the metadata.

# Connect to the *LOCAL* MySQL database
conn = mysql.connector.connect(
    host='localhost',
    port=3306,
    user='root',
    password=db_password,
    database='DRIFTBASE'
)

cursor = conn.cursor()

# Collect the stock_id from the STOCK table using the symbol.
cursor.execute(f"SELECT stock_id FROM STOCK WHERE ticker='{symbol}';")
stock_id = cursor.fetchone()[0]

df['stock_id'] = stock_id

In [ ]:
insert_query = """
INSERT IGNORE INTO HISTORIC_PRICE (`stock_id`, `interval`, `close_timestamp`, `open_price`, `high_price`, `low_price`, `close_price`, `volume`)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
"""

records = df[['stock_id', 'interval', 'close_timestamp', 'open_price', 'high_price', 'low_price', 'close_price', 'volume']].values.tolist()

cursor.executemany(insert_query, records)
conn.commit()
conn.close()